In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier

/home/francisco/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Introducción

Las características usadas a la hora de entrenar un modelo de aprendizaje tienen un fuerte impacto en el rendimiento que podemos lograr. Las características irrelevantes pueden influir negativamente en el rendimiento de nuestro algoritmo.

# Selección de características

El proceso de selección de características es un procedimiento que nos permite seleccionar aquellas características de nuestros datos que más contribuyen a nuestra variable de predicción. Tener características irrelevantes en nuestros datos puede dar lugar a empeorar la precisión de nuestros modelos, especialmente en algoritmos lineales o en la regresión logística. Entre los diferentes beneficios que aporta la selección de caracterśiticas se encuentran:

* **Reducción Overfitting:** disponer de menos datos redundantes significa menor oportunidad de tomar decisiones basdas en ruido.

* **Mejorar Accuracy:** menos datos engañosos significa una mejora en la precisión del modelado.

* **Reducción del tiempo de entrenamiento:** menor número de características significa que los algoritmos se entrenarán más rápido.

# Selección Univariante

Los test estadísticos pueden ser usados para seleccionar aquellas características que tienen un mayor impacto en la salida de nuestro algoritmo, es decir, aquellas que tienen una mayor relación con la variable a predecir. La librería scikit-lear proporciona la clase **SelectKBest** que puede ser usada conjuntamente con diferentes test estadísticos a la hora de seleccionar un número especifico de características.

A continuación vamos a proceder a seleccionar las 4 características más importantes, haciendo uso del test estadístico chi cuadrado. Recordar que este test mira la dependencia entre variables estocásticas, por lo que el uso de esta función elimina aquellas características que tienen una mayor probabilidad  de ser independientes con la variable a predecir, y por lo tanto irrelevantes. Recordar que todos los valores deben de ser positivos.

In [19]:
#Cargamos el conjunto de datos
filename = 'pima-indians-diabetes.data.csv'
names = ['preg' , 'plas' , 'pres' , 'skin' , 'test' , 'mass' , 'pedi' , 'age' , 'class']
df = pd.read_csv(filename, names=names)

#Obtenemos los predictores y la clase 
X = df.iloc[:,0:8].values
Y = df.iloc[:,8].values

#Hacemos la extracción de características
test = SelectKBest(score_func = chi2, k = 4)
fit = test.fit(X,Y)

#Vemos el resultado de los scores
np.set_printoptions(precision=3)
print(fit.scores_)

#Hacemos el transform
features = fit.transform(X)

#Vemos el resultado
print(features[0:5,:])

[ 111.52  1411.887   17.605   53.108 2175.565  127.669    5.393  181.304]
[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 [ 89.   94.   28.1  21. ]
 [137.  168.   43.1  33. ]]


En este caso podemos ver como las columnas plas, test, mass y age son las más importantes(aquellas que coinciden con los índices de los scores más elevados).

# Eliminación recursiva de características

La eliminación recursiva de características (RFE) trabaja eliminando de forma recursiva atributos, construyendo un modelo sobre aquellos atributos que permanecen. Uso la precisión del modelo para identificar los atributos (y combinación de atributos) que contribuyen en mayor medida para predecir nuestra target. Recibe como argumento el modelo que queremos aplicar y el número de top features que queremos seleccionar

In [3]:
#Cargamos los datos
col_names = ['preg' , 'plas' , 'pres' , 'skin' , 'test' , 'mass' , 'pedi' , 'age' , 'class']
df = pd.read_csv('pima-indians-diabetes.data.csv', names = col_names)
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
#Separamos entre atributos y target
X = df.values[:, 0:8]
Y = df.values[:,8]

#Hacemos uso de RFE seleccionando el top3 de características
model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(X, Y)

#Mostramos el resultado
print("Número de características: %d" % fit.n_features_)
print("Características seleccionadas: %s" % fit.support_)
print("Ranking de características: %s" % fit.ranking_)

Número de características: 3
Características seleccionadas: [ True False False False False  True  True False]
Ranking de características: [1 2 3 5 6 1 1 4]


# Análisis de Componentes Principales

El Análisis de Componentes Principales (PCA) hace uso de algebra lineal para transformar el dataset en un dataset comprimido. Una de las propiedades de PCA es que podemos elegir el número de dimensiones que queremos que tenga nuestro conjunto de datos reducido.

In [2]:
#Cargamos los datos
col_names = ['preg' , 'plas' , 'pres' , 'skin' , 'test' , 'mass' , 'pedi' , 'age' , 'class']
df = pd.read_csv('pima-indians-diabetes.data.csv', names = col_names)
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#Separamos entre predictores y la variable a predecir
X = df.values[:, 0:8]
Y = df.values[:, 8]

#Aplicamos PCA, indicando que queremos 3 componentes principales
pca = PCA(n_components=3)
fit = pca.fit(X)

print('La varianza explicada es: {}'.format(fit.explained_variance_ratio_))
print(fit.components_)

La varianza explicada es: [0.88854663 0.06159078 0.02579012]
[[-2.02176587e-03  9.78115765e-02  1.60930503e-02  6.07566861e-02
   9.93110844e-01  1.40108085e-02  5.37167919e-04 -3.56474430e-03]
 [-2.26488861e-02 -9.72210040e-01 -1.41909330e-01  5.78614699e-02
   9.46266913e-02 -4.69729766e-02 -8.16804621e-04 -1.40168181e-01]
 [-2.24649003e-02  1.43428710e-01 -9.22467192e-01 -3.07013055e-01
   2.09773019e-02 -1.32444542e-01 -6.39983017e-04 -1.25454310e-01]]


# Importancia de Variables

Los árboles de decisión como Random Forest o Extra Trees puede ser usados para estimar la importancia de variables.

In [5]:
#Cargamos los datos
col_names = ['preg' , 'plas' , 'pres' , 'skin' , 'test' , 'mass' , 'pedi' , 'age' , 'class']
df = pd.read_csv('pima-indians-diabetes.data.csv', names = col_names)
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
#Separamos entre predictores y variable a predecir
X = df.values[:, 0:8]
Y = df.values[:, 8]

#Aplicamos Extra Tree para ver la importancia de características
model = ExtraTreesClassifier()
fit = model.fit(X, Y)

#Vemos el nivel de importancia de cada una de las características
print(fit.feature_importances_)

[0.10841641 0.23079062 0.09980075 0.07454761 0.07218341 0.141866
 0.11483153 0.15756368]
